In [17]:
### manipulando tabela do brasileirão série A a partir de 2012
# para verificação de estratégia de Poisson - versão baseado nos dados main leagues football data

import csv 
import statistics as st
import math
import numpy as np

# Distribuição de Poisson para resultados de futebol
def poisson(time1,time2,mgf_ccasa,mgs_ccasa,mgf_cfora,mgs_cfora,rodada):
    mgf_tcasa = time1['mgf_tcasa']
    mgs_tcasa = time1['mgs_tcasa']
    mgf_tfora = time2['mgf_tfora']
    mgs_tfora = time2['mgs_tfora']
    
    odd_jc = 0
    odd_je = 0
    odd_jf = 0
    
    # tratamento de zeros
    if mgf_tcasa == 0:
        mgf_tcasa = 0.1
        
    if mgs_tcasa == 0:
        mgs_tcasa = 0.1
        
    if mgf_tfora == 0:
        mgf_tfora = 0.1
        
    if mgs_tfora == 0:
        mgs_tfora = 0.1
        
    if mgf_ccasa == 0:
        mgf_ccasa = 0.1
        
    if mgs_ccasa == 0:
        mgs_ccasa = 0.1
        
    if mgf_cfora == 0:
        mgf_cfora = 0.1
        
    if mgs_cfora == 0:
        mgs_cfora = 0.1
    
    media_getc = mgf_tcasa / mgf_ccasa * mgs_tfora / mgs_cfora * mgf_ccasa
    media_getf = mgf_tfora / mgf_cfora * mgs_tcasa / mgs_ccasa * mgf_cfora

    # p/ prob. do número de gols
    probf_casa = [0,0,0,0,0,0,0,0,0,0,0]
    probf_fora = [0,0,0,0,0,0,0,0,0,0,0]
    probs_casa = [0,0,0,0,0,0,0,0,0,0,0]
    probs_fora = [0,0,0,0,0,0,0,0,0,0,0]

    # p/ prob. de vitória, empate e derrota
    probv_casa = 0
    probv_fora = 0
    prob_empate = 0

    # p/ prob. dupla chance
    dupla_casa = 0
    dupla_fora = 0

    # p/ prob. empate anula
    emp_ac = 0
    emp_af = 0

    # p/ prob. under e over
    under_05 = 0
    over_05 = 0
    under_15 = 0
    over_15 = 0
    under_25 = 0
    over_25 = 0
    under_35 = 0
    over_35 = 0

    # p/ prob. ambas marcam
    ambas_sim = 0
    ambas_nao = 0
    
    # criando uma matriz em python
    prob_placar = np.zeros((11,11), dtype=np.float64)
    placar_ordenado = []
    pos_placar_ordenado = []
    posicao = 0
    cont = 0

    # prob. de gols do time da casa e fora
    for i in range(11):
        probf_casa[i] = (math.exp(-media_getc)*media_getc**i)/math.factorial(i)
        probf_fora[i] = (math.exp(-media_getf)*media_getf**i)/math.factorial(i)

    # prob. de placares do jogo; prob. de 1x2; prob. de under e over
    # i -> num de gols do time da casa; j -> num de gols do time de fora
    for i in range(11):
        for j in range(11):
            # prob. de 1x2
            if i > j:
                probv_casa += probf_casa[i] * probf_fora[j] * 100
            elif j > i:
                probv_fora += probf_casa[i] * probf_fora[j] * 100
            else: 
                prob_empate += probf_casa[i] * probf_fora[j] * 100

            # prob. de under e over
            # under/over 0.5
            if i + j <= 0:
                under_05 += probf_casa[i] * probf_fora[j] * 100

            # under/over 1.5
            if i + j <= 1:
                under_15 += probf_casa[i] * probf_fora[j] * 100

            # under/over 2.5
            if i + j <= 2:
                under_25 += probf_casa[i] * probf_fora[j] * 100

            # under/over 3.5
            if i + j <= 3:
                under_35 += probf_casa[i] * probf_fora[j] * 100

            # prob. ambas marcam
            if (i > 0) and (j > 0):
                ambas_sim += probf_casa[i] * probf_fora[j] * 100

            # prob. de placar exato
            prob_placar[i][j] = probf_casa[i] * probf_fora[j] * 100
            placar_ordenado.append(prob_placar[i][j])
            pos_placar_ordenado.append(placar_ordenado[cont])
            cont += 1
            # print(f'{i} x {j} -> {prob_placar[i][j]:.2f}%')

    # prob. dupla chance
    dupla_casa = (probv_casa + prob_empate) 
    dupla_fora = (probv_fora + prob_empate)

    # prob. empate anula
    emp_ac = probv_casa/(probv_casa + probv_fora) * 100
    emp_af = 100 - emp_ac

    # prob. over 0.5
    over_05 = 100 - under_05

    # prob. over 1.5
    over_15 = 100 - under_15

    # prob. over 2.5
    over_25 = 100 - under_25

    # prob. over 3.5
    over_35 = 100 - under_35

    # prob. ambas marcam não
    ambas_nao = 100 - ambas_sim

    placar_ordenado.sort(reverse = True)
    
    
    odd_jc = 100/probv_casa
    odd_je = 100/prob_empate
    odd_jf = 100/probv_fora
    #print(f'Rodada {rodada}') 
    
    return odd_jc,odd_je,odd_jf
    
    
    '''
    print('{} x {}'.format(time1['time'], time2['time']))
    print(f'prob. de vitória casa => {probv_casa:.2f}%')
    print(f'prob. de empate: {prob_empate:.2f}%')
    print(f'prob. de vitória visitante => {probv_fora:.2f}%\n')
    

    print(f'\nPlacar Ordenado\n')
    # for i in range(121): 121 é o máximo. Nesse range pode-se escolher quantos resultados corretos exibir
    for i in range(10):
        posicao = pos_placar_ordenado.index(placar_ordenado[i])

        # Resultados Corretos
        print(f'{int(posicao/11)} x {posicao%11} -> {placar_ordenado[i]:.2f}%')

    # 1x2
    print(f'\nprob. de vitória casa => {probv_casa:.2f}%')
    print(f'prob. de vitória visitante => {probv_fora:.2f}%')
    print(f'prob. de empate: {prob_empate:.2f}%')

    # dupla chance
    print(f'\nprob. dupla chance casa => {dupla_casa:.2f}%')
    print(f'prob. dupla chance visitante => {dupla_fora:.2f}%')

    # empate anula
    print(f'\nprob. empate anula casa => {emp_ac:.2f}%')
    print(f'prob. empate anula visitante: => {emp_af:.2f}%')

    # under/over 0.5
    print(f'\nprob. under 0.5 => {under_05:.2f}%')
    print(f'prob. over 0.5 => {over_05:.2f}%')

    # under/over 1.5
    print(f'\nprob. under 1.5 => {under_15:.2f}%')
    print(f'prob. over 1.5 => {over_15:.2f}%')

    # under/over 2.5
    print(f'\nprob. under 2.5 => {under_25:.2f}%')
    print(f'prob. over 2.5 => {over_25:.2f}%')

    # under/over 3.5
    print(f'\nprob. under 3.5 => {under_35:.2f}%')
    print(f'prob. over 3.5 => {over_35:.2f}%')

    # ambas marcam
    print(f'\nprob. ambas marcam sim => {ambas_sim:.2f}%')
    print(f'prob. ambas marcam não => {ambas_nao:.2f}%')'''
    pass


def ano_dados(camp):
    # base de dados
    time_casa = ''
    time_fora = ''
    num_jogos = 0
    num_gols = 0
    media_gols = 0

    # para média de gols dos times
    mgf_tcasa = 0
    mgs_tcasa = 0
    mgf_tfora = 0
    mgs_tfora = 0

    # para média de gols do campeonato
    # antes da rodada em questão
    tgf_ccasa_ant = 0
    tgf_cfora_ant = 0
    mgf_ccasa_ant = 0
    mgs_ccasa_ant = 0
    mgf_cfora_ant = 0
    mgs_cfora_ant = 0

    # depois da rodada em questão
    mgf_ccasa = 0
    mgs_ccasa = 0
    mgf_cfora = 0
    mgs_cfora = 0

    time_ant = [''] * 20 # recebe dados do time antes do jogo da rodada
    time = [''] * 20     # recebe dados do time depois de ser realizado o jogo da rodada em quetão

    tgf_ccasa = 0
    tgf_cfora = 0

    cont1 = 0
    cont_jogos = 0
    rodada = 1
    cont_geral = 1
    cont_ind = [0] * 20

    time1_ant = ''
    time2_ant = ''
    time1 = ''
    time2 = ''
    dentro_rodada = 0

    odd_pca = 0
    odd_pea = 0
    odd_pfa = 0

    ojc = 0
    oje = 0
    ojf = 0

    resultado = ''
    saldo = 0
    num_entradas = 0

    taxa_1x0 = 0
    taxa_n1x0 = 0
    total_ja = 0

    # últimos resultados
    urc = [''] * 20
    urf = [''] * 20
    
    urc_ant = [''] * 20
    urf_ant = [''] * 20
  
    for i in range(20):
        urc[i] = [''] * 6
        urf[i] = [''] * 6
        urc_ant[i] = [''] * 6
        urf_ant[i] = [''] * 6
        
    vitc = 0
    empc = 0
    derc = 0
    vitf = 0
    derf = 0
    
    # reds seguidos
    rs = 0
    mrs = 0
    
    with open(camp, 'r', encoding = 'utf-8', newline = '') as al:
        arq1 = csv.DictReader(al)
        for linha in arq1:
            try:
            
                # odd match odds Pinnacle
                odd_pca = float(linha['PSH'])
                odd_pea = float(linha['PSD'])
                odd_pfa = float(linha['PSA']) 

                # resultado do jogo
                resultado = linha['FTR']

                # média de gols do campeonato antes da rodada em questão
                tgf_ccasa_ant = tgf_ccasa
                tgf_cfora_ant = tgf_cfora
                mgf_ccasa_ant = mgf_ccasa
                mgs_ccasa_ant = mgs_ccasa
                mgf_cfora_ant = mgf_cfora
                mgs_cfora_ant = mgs_cfora

                # média de gols do campeonato depois da rodada em questão
                tgf_ccasa += float(linha['FTHG'])
                tgf_cfora += float(linha['FTAG'])
                mgf_ccasa = tgf_ccasa/cont_geral
                mgs_ccasa = tgf_cfora/cont_geral
                mgf_cfora = mgs_ccasa
                mgs_cfora = mgf_ccasa

                # carregando os times e o número de jogos de cada
                if cont1 < 10:
                    cont_ind[cont1] += 1
                    cont_ind[cont1+10] += 1
                    urc[cont1][0] = linha['HomeTeam']
                    urc[cont1+10][0] = linha['AwayTeam']

                    urf[cont1][0] = linha['HomeTeam']
                    urf[cont1+10][0] = linha['AwayTeam']

                    time[cont1] = {'time':linha['HomeTeam'],'num_jogosc':cont_ind[cont1],'num_jogosf':0,'tgf_tcasa':int(linha['FTHG']),'tgs_tcasa':int(linha['FTAG']),'tgf_tfora':0,'tgs_tfora':0,'mgf_tcasa':float(linha['FTHG'])/cont_ind[cont1],'mgs_tcasa':float(linha['FTAG'])/cont_ind[cont1],'mgf_tfora':0,'mgs_tfora':0}
                    time[cont1+10] = {'time':linha['AwayTeam'],'num_jogosc':0,'num_jogosf':cont_ind[cont1+10],'tgf_tcasa':0,'tgs_tcasa':0,'tgf_tfora':int(linha['FTAG']),'tgs_tfora':int(linha['FTHG']),'mgf_tcasa':0,'mgs_tcasa':0,'mgf_tfora':float(linha['FTAG'])/cont_ind[cont1+10],'mgs_tfora':float(linha['FTHG'])/cont_ind[cont1+10]}
                    cont1 += 1                  

                # contando jogos
                if cont_jogos <= 10:
                    cont_jogos += 1

                # atualizando os dados dos times por jogo
                if cont_geral >= 10:     
                    for i in range(20):
                        if linha['HomeTeam'] == time[i]['time']: 
                            time_ant[i] = time[i].copy() # time antes da rodada em questão
                            urc_ant[i] = urc[i].copy()
                            urc[i].insert(6,resultado)
                            del urc[i][1]

                            # atualização dos times após os jogos da rodada
                            time[i]['num_jogosc'] += 1
                            time[i]['tgf_tcasa'] += int(linha['FTHG'])
                            time[i]['tgs_tcasa'] += int(linha['FTAG'])

                            time[i]['mgf_tcasa'] = time[i]['tgf_tcasa']/time[i]['num_jogosc']
                            time[i]['mgs_tcasa'] = time[i]['tgs_tcasa']/time[i]['num_jogosc']

                        if linha['AwayTeam'] == time[i]['time']:
                            time_ant[i] = time[i].copy() # time antes da rodada em questão
                            urf_ant[i] = urf[i].copy()
                            urf[i].insert(6,resultado)
                            del urf[i][1]

                            # atualização dos times após os jogos da rodada
                            time[i]['num_jogosf'] += 1
                            time[i]['tgf_tfora'] += int(linha['FTAG'])
                            time[i]['tgs_tfora'] += int(linha['FTHG'])

                            time[i]['mgf_tfora'] = time[i]['tgf_tfora']/time[i]['num_jogosf']
                            time[i]['mgs_tfora'] = time[i]['tgs_tfora']/time[i]['num_jogosf']


                #contando rodadas - a primeira vez que entra aqui é a rodada 2
                if cont_jogos > 10 or dentro_rodada < 10:
                    # atualizando o número de rodadas
                    if cont_jogos > 10:
                        rodada += 1
                        cont_jogos = 1
                        dentro_rodada = 0



                    dentro_rodada += 1

                    #carregando times com dados da rodada para chamar o método
                    if rodada > 1:
                        for i in range(20):                 
                            if linha['HomeTeam'] == time[i]['time']:
                                time1_ant = time_ant[i] # time1 antes da rodada em questão
                                time1 = time[i]         # time1 depois da rodada em questão

                                # últimos 5 resultados de cada time em casa
                                for j in range(1,6):
                                    if urc_ant[i][j] == 'H':
                                        vitc += 1
                                    elif urc_ant[i][j] == 'A':
                                        derc += 1
                                    elif urc_ant[i][j] == 'D':
                                        empc += 1


                            if linha['AwayTeam'] == time[i]['time']:
                                time2_ant = time_ant[i] # time2 antes da rodada em questão
                                time2 = time[i]         # time2 depois da rodada em questão

                                # últimos 5 resultados de cada time fora de casa
                                for j in range(1,6):
                                    if urf_ant[i][j] == 'A':
                                        vitf += 1
                                    elif urf_ant[i][j] == 'H':
                                        derf += 1
                                    elif urf_ant[i][j] == 'D':
                                        empf += 1


                    #print('número de jogos: {}'.format(cont_geral))
                    # chamando o método para cálculo de probabilidades
                    if rodada > 0: 
                        #ojc,oje,ojf = poisson(time1_ant,time2_ant,mgf_ccasa_ant,mgs_ccasa_ant,mgf_cfora_ant,mgs_cfora_ant,rodada)
                        if odd_pca >= 1.18 and odd_pca <= 1.43:
                            num_entradas += 1
                            if resultado == 'H':
                                if rs > mrs:
                                    mrs = rs
                                rs = 0
                                print(f'Rodada: {rodada}')
                                print('equipes: {} x {}'.format(linha['HomeTeam'],linha['AwayTeam']))
                                print(f'Green back com odd: {odd_pca}')
                                print('Entradas: {}  Saldo: {s:.2f}\n'.format(num_entradas,s = saldo))
                                saldo = saldo + (2000 * (odd_pca - 1))
                            else:
                                print(f'Rodada: {rodada}')
                                print(f'Red no back com odd: {odd_pca}')
                                print('Entradas: {}  Saldo: {s:.2f}\n'.format(num_entradas,s = saldo))
                                rs += 1                  
                                saldo = saldo - 2000

                            '''elif derf <= 1 (((odd_pfa / ojf) - 1) * 100) >= 20:
                                num_entradas += 1
                                if (linha['FTR'] == 'A'):
                                    if rs > mrs:
                                        mrs = rs
                                    rs = 0
                                    saldo += 5 * (odd_pfa - 1)
                                else:
                                    rs += 1
                                    saldo -= 5'''
                            pass

                cont_geral += 1
                vitc = 0
                empc = 0
                derc = 0
                vitf = 0
                derf = 0
                empf = 0
            except:
                continue
           
    print(f'Campeonato: {camp}')
    print(f'Máximo de reds seguidos: {mrs}')
    print(f'Total de partidas da temporada: {cont_geral - 1}')
    print('Entradas: {}  Saldo: {s:.2f}'.format(num_entradas,s = saldo))
    print('Banca fim de temporada {s:.2f}\n'.format(s = saldo))
    return num_entradas,saldo,cont_geral-1
    
    
    
# programa principal
# ano a ser avaliado
ent2 = input('Campeonato: ')
entrada2 = ent2.upper().split(',') # campeonato -> entrada2
tam_ent2 = len(entrada2)
print('\n')
entrada2 = entrada2

for i in range(tam_ent2):
    entrada2[i] +='.csv'


saldo_final = 100
quant_entradas = 0
cont_geral = 0
fcont_geral = 0

for i in range(tam_ent2):
    try:
        quant_ent,saldo,cont_geral = ano_dados(entrada2[i])
        fcont_geral += cont_geral
        quant_entradas += quant_ent
        saldo_final += saldo
    except:
        continue

print(f'Total de jogos: {fcont_geral}')
print(f'Número de entradas: {quant_entradas}   Saldo final: {saldo_final:.2f}')
    

Campeonato: s1,s2,s3,s4,s5,s6,e1,e2,e3,e4,e5,e6,e7,e8,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,f1,f2,f3,f4,f5,f6,f7,f8,f9,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,p1,p2,p3,p4,p5,p6,p7,p8,p9


Rodada: 1
equipes: Real Madrid x Getafe
Green back com odd: 1.2
Entradas: 1  Saldo: 0.00

Rodada: 4
Red no back com odd: 1.36
Entradas: 2  Saldo: 400.00

Rodada: 5
equipes: Real Madrid x Espanol
Green back com odd: 1.2
Entradas: 3  Saldo: -1600.00

Rodada: 5
equipes: Ath Madrid x Huesca
Green back com odd: 1.24
Entradas: 4  Saldo: -1200.00

Rodada: 9
equipes: Barcelona x Sevilla
Green back com odd: 1.35
Entradas: 5  Saldo: -720.00

Rodada: 10
equipes: Sevilla x Huesca
Green back com odd: 1.25
Entradas: 6  Saldo: -20.00

Rodada: 11
equipes: Real Madrid x Valladolid
Green back com odd: 1.2
Entradas: 7  Saldo: 480.00

Rodada: 12
Red no back com odd: 1.22
Entradas: 8  Saldo: 880.00

Rodada: 13
equipes: Valencia x Vallecano
Green back com odd: 1.43
Entradas: 9  Saldo: -1120.00

Rodada: 13
eq

equipes: Ath Madrid x Elche
Green back com odd: 1.21
Entradas: 36  Saldo: 10240.00

Rodada: 33
equipes: Sevilla x Vallecano
Green back com odd: 1.38
Entradas: 37  Saldo: 10660.00

Rodada: 33
equipes: Valencia x Granada
Green back com odd: 1.32
Entradas: 38  Saldo: 11420.00

Rodada: 35
equipes: Valencia x Eibar
Green back com odd: 1.23
Entradas: 39  Saldo: 12060.00

Rodada: 36
equipes: Villarreal x Elche
Green back com odd: 1.37
Entradas: 40  Saldo: 12520.00

Rodada: 37
equipes: Sevilla x Almeria
Green back com odd: 1.29
Entradas: 41  Saldo: 13260.00

Rodada: 38
Red no back com odd: 1.29
Entradas: 42  Saldo: 13840.00

Rodada: 38
equipes: Eibar x Cordoba
Green back com odd: 1.38
Entradas: 43  Saldo: 11840.00

Campeonato: S5.csv
Máximo de reds seguidos: 1
Total de partidas da temporada: 378
Entradas: 43  Saldo: 12600.00
Banca fim de temporada 12600.00

Rodada: 2
equipes: Ath Madrid x Vallecano
Green back com odd: 1.32
Entradas: 1  Saldo: 0.00

Rodada: 3
equipes: Real Madrid x Ath Bilbao
G

Rodada: 21
equipes: Liverpool x Leicester
Green back com odd: 1.32
Entradas: 39  Saldo: -960.00

Rodada: 21
Red no back com odd: 1.36
Entradas: 40  Saldo: -320.00

Rodada: 22
equipes: Man City x Watford
Green back com odd: 1.18
Entradas: 41  Saldo: -2320.00

Rodada: 22
Red no back com odd: 1.26
Entradas: 42  Saldo: -1960.00

Rodada: 23
Red no back com odd: 1.3
Entradas: 43  Saldo: -3960.00

Rodada: 23
equipes: Tottenham x Everton
Green back com odd: 1.28
Entradas: 44  Saldo: -5960.00

Rodada: 23
equipes: Man United x Stoke
Green back com odd: 1.23
Entradas: 45  Saldo: -5400.00

Rodada: 25
Red no back com odd: 1.3
Entradas: 46  Saldo: -4940.00

Rodada: 26
equipes: Arsenal x Everton
Green back com odd: 1.32
Entradas: 47  Saldo: -6940.00

Rodada: 27
equipes: Man City x Leicester
Green back com odd: 1.22
Entradas: 48  Saldo: -6300.00

Rodada: 27
equipes: Chelsea x West Brom
Green back com odd: 1.4
Entradas: 49  Saldo: -5860.00

Rodada: 28
equipes: Liverpool x West Ham
Green back com odd: 1

Rodada: 20
Red no back com odd: 1.42
Entradas: 23  Saldo: 2500.00

Rodada: 20
equipes: Man City x Sunderland
Green back com odd: 1.25
Entradas: 24  Saldo: 500.00

Rodada: 21
equipes: Chelsea x Newcastle
Green back com odd: 1.2
Entradas: 25  Saldo: 1000.00

Rodada: 23
equipes: Man United x Leicester
Green back com odd: 1.36
Entradas: 26  Saldo: 1400.00

Rodada: 23
equipes: Arsenal x Aston Villa
Green back com odd: 1.27
Entradas: 27  Saldo: 2120.00

Rodada: 24
Red no back com odd: 1.19
Entradas: 28  Saldo: 2660.00

Rodada: 25
equipes: Arsenal x Leicester
Green back com odd: 1.29
Entradas: 29  Saldo: 660.00

Rodada: 25
equipes: Chelsea x Everton
Green back com odd: 1.33
Entradas: 30  Saldo: 1240.00

Rodada: 25
equipes: Man United x Burnley
Green back com odd: 1.31
Entradas: 31  Saldo: 1900.00

Rodada: 26
Red no back com odd: 1.21
Entradas: 32  Saldo: 2520.00

Rodada: 26
equipes: Man City x Newcastle
Green back com odd: 1.3
Entradas: 33  Saldo: 520.00

Rodada: 27
equipes: Man United x Sund

Rodada: 11
Red no back com odd: 1.39
Entradas: 15  Saldo: -980.00

Rodada: 11
equipes: Fenerbahce x Alanyaspor
Green back com odd: 1.43
Entradas: 16  Saldo: -2980.00

Rodada: 16
equipes: Galatasaray x Ankaragucu
Green back com odd: 1.3
Entradas: 17  Saldo: -2120.00

Rodada: 17
Red no back com odd: 1.4
Entradas: 18  Saldo: -1520.00

Rodada: 18
equipes: Trabzonspor x Ankaragucu
Green back com odd: 1.3
Entradas: 19  Saldo: -3520.00

Rodada: 19
Red no back com odd: 1.3
Entradas: 20  Saldo: -2920.00

Rodada: 21
equipes: Galatasaray x Akhisar Belediyespor
Green back com odd: 1.3
Entradas: 21  Saldo: -4920.00

Rodada: 23
equipes: Galatasaray x Antalyaspor
Green back com odd: 1.3
Entradas: 22  Saldo: -4320.00

Rodada: 24
equipes: Galatasaray x Yeni Malatyaspor
Green back com odd: 1.43
Entradas: 23  Saldo: -3720.00

Rodada: 24
equipes: Buyuksehyr x Konyaspor
Green back com odd: 1.43
Entradas: 24  Saldo: -2860.00

Rodada: 26
equipes: Galatasaray x Kayserispor
Green back com odd: 1.34
Entradas: 2

...

Rodada: 7
equipes: Anderlecht x Lokeren
Green back com odd: 1.43
Entradas: 10  Saldo: -2080.00

Rodada: 9
equipes: Anderlecht x Waasland-Beveren
Green back com odd: 1.24
Entradas: 11  Saldo: -1220.00

Rodada: 13
equipes: Anderlecht x Kortrijk
Green back com odd: 1.33
Entradas: 12  Saldo: -740.00

Rodada: 13
equipes: Standard x Lierse
Green back com odd: 1.43
Entradas: 13  Saldo: -80.00

Rodada: 14
equipes: Club Brugge x Charleroi
Green back com odd: 1.37
Entradas: 14  Saldo: 780.00

Rodada: 14
equipes: Anderlecht x Cercle Brugge
Green back com odd: 1.3
Entradas: 15  Saldo: 1520.00

Rodada: 15
equipes: Standard x Charleroi
Green back com odd: 1.36
Entradas: 16  Saldo: 2120.00

Rodada: 16
equipes: Genk x Lierse
Green back com odd: 1.38
Entradas: 17  Saldo: 2840.00

Rodada: 17
Red no back com odd: 1.41
Entradas: 18  Saldo: 3600.00

Rodada: 18
equipes: Standard x Germinal
Green back com odd: 1.28
Entradas: 19  Saldo: 1600.00

Rodada: 18
equipes: Genk x Charleroi
Green back com odd: 1.4
Ent

equipes: Monaco x Toulouse
Green back com odd: 1.27
Entradas: 25  Saldo: -1600.00

Rodada: 37
equipes: Monaco x Lille
Green back com odd: 1.19
Entradas: 26  Saldo: -1060.00

Rodada: 37
equipes: Monaco x St Etienne
Green back com odd: 1.22
Entradas: 27  Saldo: -680.00

Rodada: 38
equipes: Marseille x Bastia
Green back com odd: 1.32
Entradas: 28  Saldo: -240.00

Rodada: 38
Red no back com odd: 1.2
Entradas: 29  Saldo: 400.00

Campeonato: F3.csv
Máximo de reds seguidos: 2
Total de partidas da temporada: 378
Entradas: 29  Saldo: -1600.00
Banca fim de temporada -1600.00

Rodada: 5
Red no back com odd: 1.28
Entradas: 1  Saldo: 0.00

Rodada: 7
equipes: Paris SG x Guingamp
Green back com odd: 1.21
Entradas: 2  Saldo: -2000.00

Rodada: 9
equipes: Paris SG x Marseille
Green back com odd: 1.36
Entradas: 3  Saldo: -1580.00

Rodada: 11
equipes: Paris SG x St Etienne
Green back com odd: 1.28
Entradas: 4  Saldo: -860.00

Rodada: 13
equipes: Paris SG x Toulouse
Green back com odd: 1.19
Entradas: 5  Sa

Entradas: 3  Saldo: 1540.00

Rodada: 3
Red no back com odd: 1.4
Entradas: 4  Saldo: 2060.00

Rodada: 3
Red no back com odd: 1.21
Entradas: 5  Saldo: 60.00

Rodada: 3
Red no back com odd: 1.3
Entradas: 6  Saldo: -1940.00

Rodada: 3
equipes: PAOK x Iraklis
Green back com odd: 1.22
Entradas: 7  Saldo: -3940.00

Rodada: 5
equipes: Panathinaikos x Iraklis
Green back com odd: 1.21
Entradas: 8  Saldo: -3500.00

Rodada: 6
equipes: AEK x Platanias
Green back com odd: 1.34
Entradas: 9  Saldo: -3080.00

Rodada: 6
equipes: Giannina x Veria
Green back com odd: 1.41
Entradas: 10  Saldo: -2400.00

Rodada: 7
Red no back com odd: 1.42
Entradas: 11  Saldo: -1580.00

Rodada: 7
equipes: AEK x Levadeiakos
Green back com odd: 1.29
Entradas: 12  Saldo: -3580.00

Rodada: 7
Red no back com odd: 1.31
Entradas: 13  Saldo: -3000.00

Rodada: 8
equipes: PAOK x Platanias
Green back com odd: 1.34
Entradas: 14  Saldo: -5000.00

Rodada: 9
equipes: Olympiakos x Panetolikos
Green back com odd: 1.19
Entradas: 15  Saldo: -

Green back com odd: 1.18
Entradas: 2  Saldo: 500.00

Rodada: 1
equipes: Sp Braga x Nacional
Green back com odd: 1.36
Entradas: 3  Saldo: 860.00

Rodada: 2
equipes: Sp Lisbon x Setubal
Green back com odd: 1.25
Entradas: 4  Saldo: 1580.00

Rodada: 3
equipes: Sp Braga x Aves
Green back com odd: 1.3
Entradas: 5  Saldo: 2080.00

Rodada: 4
equipes: Sp Lisbon x Feirense
Green back com odd: 1.29
Entradas: 6  Saldo: 2680.00

Rodada: 5
equipes: Sp Lisbon x Maritimo
Green back com odd: 1.36
Entradas: 7  Saldo: 3260.00

Rodada: 6
Red no back com odd: 1.4
Entradas: 8  Saldo: 3980.00

Rodada: 8
equipes: Sp Lisbon x Boavista
Green back com odd: 1.33
Entradas: 9  Saldo: 1980.00

Rodada: 8
equipes: Sp Braga x Setubal
Green back com odd: 1.42
Entradas: 10  Saldo: 2640.00

Rodada: 9
equipes: Porto x Sp Braga
Green back com odd: 1.4
Entradas: 11  Saldo: 3480.00

Rodada: 9
equipes: Sp Lisbon x Chaves
Green back com odd: 1.32
Entradas: 12  Saldo: 4280.00

Rodada: 10
equipes: Sp Braga x Moreirense
Green back

Rodada: 8
Red no back com odd: 1.23
Entradas: 12  Saldo: 1260.00

Rodada: 9
equipes: Sp Braga x Olhanense
Green back com odd: 1.3
Entradas: 13  Saldo: -740.00

Rodada: 10
equipes: Sp Braga x Setubal
Green back com odd: 1.38
Entradas: 14  Saldo: -140.00

Rodada: 10
equipes: Sp Lisbon x Belenenses
Green back com odd: 1.38
Entradas: 15  Saldo: 620.00

Rodada: 11
Red no back com odd: 1.43
Entradas: 16  Saldo: 1380.00

Rodada: 13
equipes: Benfica x Maritimo
Green back com odd: 1.24
Entradas: 17  Saldo: -620.00

Rodada: 14
Red no back com odd: 1.38
Entradas: 18  Saldo: -140.00

Rodada: 15
equipes: Sp Braga x Gil Vicente
Green back com odd: 1.32
Entradas: 19  Saldo: -2140.00

Rodada: 16
equipes: Benfica x Guimaraes
Green back com odd: 1.27
Entradas: 20  Saldo: -1500.00

Rodada: 16
equipes: Estoril x Olhanense
Green back com odd: 1.34
Entradas: 21  Saldo: -960.00

Rodada: 18
equipes: Porto x Arouca
Green back com odd: 1.19
Entradas: 22  Saldo: -280.00

Rodada: 19
equipes: Benfica x Academica
G

In [ ]:
#t1,t2,t3,t4,t5,t6,t7,t8,t9,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,e1,e2,e3,e4,e5,e6,e7,ec1,ec2,ec3,ec4,ec5,ec6,ec7,ec8,ec9,f1,f2,f3,f4,f5,f6,f7,f8,f9,g,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,i1,i2,i3,i4,i5,i6,n1,n2,n3,n4,n5,n6,n7,n8,n9,p1,p2,p3,p4,p5,p6,p7,p8,p9,s1,s2,s3,s4,s5,s6,sc1,sc2,sc3,sc4,sc5,sc6,sc7,sc8,sc9

In [ ]:
# t1,b1,d1,e1,ec1,f1,g1,i1,n1,p1,s1,sc1

In [ ]:
# s1,s2,s3,s4,s5,s6,d1,d2,d3,d4,d5,d6,i1,i2,i3,i4,i5,i6,e1,e2,e3,e4,e5,e6